# Import

In [12]:
import numpy as np
#%pylab inline
import pandas as pd
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
import catboost as cb
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import copy
from sklearn.utils import shuffle
import os
import requests
import json
from tqdm import tqdm_notebook as tqdm
warnings.filterwarnings('ignore')

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import data 

In [14]:
df_train = pd.read_csv('sberbank_data/data_2019/dota2_skill_train.csv')
df_test = pd.read_csv('sberbank_data/data_2019/dota2_skill_test.csv')
train_sib = pd.read_csv('sberbank_data/skill_train.csv')
test_index = df_test['id']
df_test = df_test.drop('id', axis=1)

## Find data-leaks 

In [15]:
pd.merge(train_sib, df_test, on=[*df_test.columns], how='inner')

,id,skilled,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,...,avg_deaths_x16,avg_assists_x16,avg_gpm_x16,avg_xpm_x16,best_kills_x16,best_assists_x16,best_gpm_x16,best_xpm_x16,win_streak,best_win_streak


## Concat sberbank and vsesib dataset 

In [16]:
#df_train = pd.concat([df_train,train_sib])
df_train = df_train.drop('id',axis=1)
df_train = df_train.reset_index(drop=True)

#train_sib = train_sib.drop('id',axis=1)
#train_sib = train_sib.reset_index(drop=True)

In [17]:
df_train.shape

(99871, 57)

## Overview

In [18]:
df_train.head(2)

,skilled,player_team,winner_team,duration,pre_game_duration,first_blood_time,first_blood_claimed,hero_id,hero_pick_order,leaver_status,...,avg_deaths_x16,avg_assists_x16,avg_gpm_x16,avg_xpm_x16,best_kills_x16,best_assists_x16,best_gpm_x16,best_xpm_x16,win_streak,best_win_streak
0,1,dire,dire,2140,90,129,0,90,9,0,...,8,15,352,430,10,30,551,745,2,5
1,0,radiant,radiant,2138,90,174,0,5,5,0,...,9,19,294,425,13,37,445,717,2,12


## Data from JSON 

In [19]:
#JSON path
path_vsesib = 'sberbank_data/skill_train.jsonlines'
path_2019 = 'sberbank_data/data_2019/dota2_skill_train.jsonlines'
path_test = 'sberbank_data/data_2019/dota2_skill_test.jsonlines'

In [20]:
train_paths = [path_2019]

## Train data 

In [21]:
from source.functions_v2 import *

In [22]:
buff = []
ids = []

data_gold = []
data_items = []
data_damage = []
data_abilities = []
data_lvls = []
batch_size = 5000
for path in train_paths:
    with open(path) as inp:
        for i, line in tqdm(enumerate(inp)):
            record = json.loads(line)
            buff.append(record)
            ids.append(record['id'])

            if (i + 1) % batch_size == 0:
                # extract features
                data_gold.append(get_gold_features(buff))
                data_items.append(get_items_features(buff))
                data_abilities.append(get_abilities(buff))
                data_damage.append(get_damage_features(buff))
                data_lvls.append(get_lvl_times(buff))
                # free buff
                buff = []

        # process last batch
        data_gold.append(get_gold_features(buff))
        data_items.append(get_items_features(buff))
        data_abilities.append(get_abilities(buff))
        data_damage.append(get_damage_features(buff))
        data_lvls.append(get_lvl_times(buff))

        # free buff
        buff = []

A Jupyter Widget

In [23]:
gold = np.concatenate(data_gold)
items = np.concatenate(data_items)
damage = np.concatenate(data_damage)
abilities = np.concatenate(data_abilities)
levels = np.concatenate(data_lvls)

In [24]:
extra_data = np.concatenate([gold,items,damage,abilities,levels],axis=1)

In [25]:
extra_data_df = pd.DataFrame(data=extra_data,columns=columns_gold+columns_items+columns_damage+columns_abililies+columns_lvl)

In [26]:
full_df = pd.concat([df_train, extra_data_df],axis=1)

In [27]:
full_df.shape

(99871, 650)

## Test data

In [28]:
test_path = 'sberbank_data/data_2019/dota2_skill_test.jsonlines'

In [29]:
test_buff = []
test_ids = []

test_data_gold = []
test_data_items = []
test_data_damage = []
test_data_abilities = []
test_data_lvls = []
batch_size = 5000

with open(test_path) as inp:
    for i, line in tqdm(enumerate(inp)):
        record = json.loads(line)
        test_buff.append(record)
        test_ids.append(record['id'])
        
        if (i + 1) % batch_size == 0:
            # extract features
            test_data_gold.append(get_gold_features(test_buff))
            test_data_items.append(get_items_features(test_buff))
            test_data_abilities.append(get_abilities(test_buff))
            test_data_damage.append(get_damage_features(test_buff))
            
            test_data_lvls.append(get_lvl_times(test_buff))
            # free buff
            test_buff = []
    
    # process last batch
    test_data_gold.append(get_gold_features(test_buff))
    test_data_items.append(get_items_features(test_buff))
    test_data_abilities.append(get_abilities(test_buff))
    test_data_damage.append(get_damage_features(test_buff))
    
    test_data_lvls.append(get_lvl_times(test_buff))

    # free buff
    test_buff = []

A Jupyter Widget

In [30]:
test_gold = np.concatenate(test_data_gold)
test_items = np.concatenate(test_data_items)
test_damage = np.concatenate(test_data_damage)
test_abilities = np.concatenate(test_data_abilities)
test_levels = np.concatenate(test_data_lvls)

In [31]:
test_extra_data = np.concatenate([test_gold,test_items,test_damage,test_abilities,test_levels],axis=1)

In [32]:
extra_test_df = pd.DataFrame(data=test_extra_data,columns=columns_gold+columns_items+columns_damage+columns_abililies+columns_lvl)

In [33]:
full_test = pd.concat([df_test, extra_test_df],axis=1)

In [34]:
full_test.shape, full_df.shape

((43265, 649), (99871, 650))

In [35]:
# get dummies on 'hero_id', 'player_team' and 'winner_team'
hero_dummies = pd.get_dummies(full_df['hero_id'], prefix='hero_id')
player_team = pd.get_dummies(full_df['player_team'], prefix='player_team')
winner_team = pd.get_dummies(full_df['winner_team'], prefix='winner_team')

# drop colums
full_df.drop(['hero_id'], inplace=True, axis=1)
full_df.drop(['player_team'], inplace=True, axis=1)
full_df.drop(['winner_team'], inplace=True, axis=1)

# concatenate them to dataframe
full_df = pd.concat([full_df, player_team, winner_team, hero_dummies], axis=1)
full_df = full_df.drop('winner_team_other', axis = 1)
full_df.shape

(99871, 768)

In [36]:
# get dummies on 'hero_id', 'player_team' and 'winner_team'
hero_dummies = pd.get_dummies(full_test['hero_id'], prefix='hero_id')
player_team = pd.get_dummies(full_test['player_team'], prefix='player_team')
winner_team = pd.get_dummies(full_test['winner_team'], prefix='winner_team')

# drop colums
full_test.drop(['hero_id'], inplace=True, axis=1)
full_test.drop(['player_team'], inplace=True, axis=1)

full_test.drop(['winner_team'], inplace=True, axis=1)

# concatenate them to dataframe
full_test = pd.concat([full_test, player_team, winner_team, hero_dummies], axis=1)


In [37]:
full_test.shape, full_df.shape

((43265, 767), (99871, 768))

## x_train and y_train

In [38]:
train = full_df.drop('skilled',axis=1)
y_train = full_df['skilled']

train = train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [39]:
train.shape

(99871, 767)

## Xgboost param 

In [29]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [42]:
def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, train.drop(to_drop, axis=1), y_train, cv=3).mean()
    print(current_score, params)
    return -current_score

space_xgb2 = {
            'n_estimators': hp.choice('n_estimators', range(100, 1200)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 18, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            # Increase this number if you have more cores. Otherwise, remove it and it will default
            # to the maxium number.
            'nthread': 8,
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1
        }
best = fmin(fn=hyperopt_xgb_score, space=space_xgb2, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

0.7661291403600488 {'booster': 'gbtree', 'colsample_bytree': 0.75, 'eta': 0.35000000000000003, 'eval_metric': 'auc', 'gamma': 0.75, 'max_depth': 16, 'min_child_weight': 2.0, 'n_estimators': 572, 'nthread': 8, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.7000000000000001, 'tree_method': 'exact'}
0.7709481106151391 {'booster': 'gbtree', 'colsample_bytree': 0.55, 'eta': 0.5, 'eval_metric': 'auc', 'gamma': 0.9500000000000001, 'max_depth': 8, 'min_child_weight': 4.0, 'n_estimators': 402, 'nthread': 8, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8, 'tree_method': 'exact'}
0.7721813380517687 {'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.125, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'max_depth': 9, 'min_child_weight': 5.0, 'n_estimators': 590, 'nthread': 8, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
0.7071688047077117 {'booster': 'gbtree', 'colsample_bytree': 0.9, 'eta':

In [40]:
best = {'colsample_bytree': 0.9500000000000001,
 'eta': 0.125,
 'gamma': 0.8500000000000001,
 'max_depth': 13,
 'min_child_weight': 5.0,
 'n_estimators': 3000,
 'subsample': 0.8500000000000001}

## LGB params 

In [31]:
from paramsearch import paramsearch
from sortedcontainers import SortedList
import copy
import collections
from itertools import product,chain
from sklearn.model_selection import KFold
import lightgbm as lgb

In [32]:
params = {'boosting_type': ['gbdt'],
 'colsample_bytree': [0.64, 0.65, 0.66],
 'learning_rate': [0.03,0.001,0.01,0.1,0.2,0.3],
'num_iterations' : [250,100,500,1000,1100,1200],
 'max_bin': 512,
 'max_depth': [3,1,2,6,4,5,7,8,9,10,12,13,14],
 'metric': 'binary_error',
 'min_child_samples': 5,
 'min_child_weight': 1,
 'min_split_gain': 0.5,
 'n_estimators': range(10,1200,80),
 'nthread': 5,
 'num_class': 1,
 'num_leaves': [6, 8, 12, 16],
 'objective': ['binary'],
 'random_state': [228],
 'reg_alpha': [1, 1.2,1.3,1.4],
 'reg_lambda': [1, 1.2, 1.4,1.5,1.6],
 'scale_pos_weight': 1,
 'subsample': [0.7, 0.75],
 'subsample_for_bin': 200,
 'subsample_freq': 1}
def crossvaltest(params,train_set,train_label,n_splits=10):
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set,train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = lgb.LGBMClassifier(**params)
        clf.fit(train, np.ravel(labels))

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)
def lightgbm_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['boosting_type']),
                      ps.grid_search(['colsample_bytree']),
                      ps.grid_search(['learning_rate','num_iterations']),
                      ps.grid_search(['n_estimators']),
                    ps.grid_search(['num_leaves']),
                     ps.grid_search(['objective']),
                     ps.grid_search(['reg_alpha']),
                     ps.grid_search(['reg_lambda']),
                     ps.grid_search(['subsample']),
                     ps.grid_search(['max_depth'])):
        res = crossvaltest(prms,train_set,train_label,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams_lgb = lightgbm_param_tune(params,train.drop(to_drop, axis=1),y_train)

0.7044208522673959 {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.03, 'num_iterations': 250, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7044208523206239 {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.03, 'num_iterations': 250, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7045683033739495 {'boosting_type': 'gb

0.5957694937065187 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.001, 'num_iterations': 500, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7462098364407026 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.03, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.6439994101955738 {'boosting_type': '

0.7186297770807362 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.01, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7462098364407026 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.03, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7372957467058082 {'boosting_type': '

0.7682135628208737 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 500, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7723354914606144 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.1, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7777241591935766 {'boosting_type': 'gbd

0.7789037680460047 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.3, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7790311121683089 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 10, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7794466562110428 {'boosting_type': 'gb

0.7773555314271926 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 730, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7800029489870098 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 6, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7793461213656654 {'boosting_type': '

0.7791584563209609 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1.2, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.7811825579789591 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7794466562110428 {'boosting_type

0.7713301430275733 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 2, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1.6, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.781397032171313 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1.6, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7794667631801181 {'boosting_typ

0.7787831262315518 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 13, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1.6, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1} best: 0.781397032171313 {'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.2, 'num_iterations': 1200, 'max_bin': 512, 'max_depth': 3, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 330, 'nthread': 5, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'random_state': 228, 'reg_alpha': 1, 'reg_lambda': 1.6, 'scale_pos_weight': 1, 'subsample': 0.7, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.7773957453653436 {'boosting_ty

## Catboost param 

In [106]:
def get_cat_index(dataframe):
    cat_features = ['player_team','winner_team','hero_id','hero_pick_order','leaver_status','party_players','winner', 'abil_1',
 'abil_2',
 'abil_3',
 'abil_4',
 'abil_5',
 'abil_6',
 'abil_7',
 'abil_8',
 'abil_9',
 'abil_10',
 'abil_11',
 'abil_12',
 'abil_13',
 'abil_14',
 'abil_15',
 'abil_16',
 'abil_17',
 'abil_18',
 'abil_19',
 'abil_20',
 'abil_21',
 'abil_22',
 'abil_23',
 'abil_24',
 'abil_25',
 'first_blood_claimed','']
    categorical_features_index = [i for i, j in  enumerate(dataframe.columns) if j in cat_features]
    
    return categorical_features_index

In [31]:
cat_feat = get_cat_index(train)

In [53]:
params = {'depth':[3,1,2,6,4,5,7,8,9,10,11,12,13],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'thread_count':8,
         'verbose':False}
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,train.drop(to_drop,axis=1),y_train,cat_feat)

0.6976515060119838 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 5, 'thread_count': 8, 'verbose': False} best: 0.6976515058569602 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 5, 'thread_count': 8, 'verbose': False}
0.7016125789198536 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 10, 'thread_count': 8, 'verbose': False} best: 0.7016125789041755 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 10, 'thread_count': 8, 'verbose': False}
0.7039248803635342 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 20, 'thread_count': 8, 'verbose': False} best: 0.7039248804280654 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 20, 'thread_count': 8, 'verbose': False}
0.7051916194152893 {'depth': 3, 'iterations': 250, 'learning_rate': 0.03, 'l2_leaf_reg': 3

0.7396482620876395 {'depth': 3, 'iterations': 1000, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False} best: 0.7630929880259812 {'depth': 3, 'iterations': 500, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False}
0.6556681545823783 {'depth': 3, 'iterations': 1000, 'learning_rate': 0.001, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False} best: 0.7630929880259812 {'depth': 3, 'iterations': 500, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False}
0.7122759748528841 {'depth': 3, 'iterations': 1000, 'learning_rate': 0.01, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False} best: 0.7630929880259812 {'depth': 3, 'iterations': 500, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False}
0.761249849197732 {'depth': 3, 'iterations': 1000, 'learning_rate': 0.1, 'l2_leaf


Iteration with suspicious time -1.95e+04 sec ignored in overall statistics.


0.7555729815954209 {'depth': 11, 'iterations': 1000, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False} best: 0.7714038684498806 {'depth': 5, 'iterations': 1000, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False}


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



## Params

In [41]:
# not drop "useless" feature
bestparams_lgb = {'boosting_type': 'gbdt',
 'colsample_bytree': 0.66,
 'learning_rate': 1e-2,
 'num_iterations': 1200,
 'max_bin': 512,
 'max_depth': 8,
 'metric': 'binary_error',
 'min_child_samples': 5,
 'min_child_weight': 1,
 'min_split_gain': 0.5,
 'n_estimators': 3000,
 'nthread': 5,
 'num_class': 1,
 'num_leaves': 16,
 'objective': 'binary',
 'random_state': 228,
 'reg_alpha': 1,
 'reg_lambda': 1.3,
 'scale_pos_weight': 1,
 'subsample': 0.7,
 'subsample_for_bin': 200,
 'subsample_freq': 1}

In [65]:
best_cb = {'depth': 10, 'iterations': 1000, 'learning_rate': 0.3, 'l2_leaf_reg': 3, 'border_count': 100, 'thread_count': 8, 'verbose': False}

In [242]:
best = {'colsample_bytree': 0.9500000000000001,
'eta': 0.125,
'gamma': 0.8500000000000001,
'max_depth': 16,
'min_child_weight': 5.0,
'n_estimators': 3000,
'subsample': 0.8500000000000001,'learning_rate':0.28,'reg_lambda':22}

## Feature, add extra-data

In [40]:
buff = []
ids = []

gold_dev = []
targets_value = []
items_value = []

trends = []
batch_size = 5000

for path in train_paths:  
    with open(path) as inp:
        for i, line in tqdm(enumerate(inp)):
            record = json.loads(line)
            buff.append(record)
            ids.append(record['id'])

            if (i + 1) % batch_size == 0:
                # extract features
                gold_dev.append(gold_deviation(buff))
                targets_value.append(get_targets_value(buff))
                items_value.append(get_items_value(buff))
                trends.append(timeseries_trend(buff))
                # free buff
                buff = []

        # process last batch
        gold_dev.append(gold_deviation(buff))
        targets_value.append(get_targets_value(buff))
        items_value.append(get_items_value(buff))
        trends.append(timeseries_trend(buff))
        


        # free buff
        buff = []

A Jupyter Widget

In [86]:
buff = []
test_ids = []

test_gold_dev = []
test_self_damage = []
test_friend_damage = []
test_targets_value = []
test_items_value = []

test_trends = []

batch_size = 5000

with open(test_path) as inp:
    for i, line in tqdm(enumerate(inp)):
        record = json.loads(line)
        buff.append(record)
        test_ids.append(record['id'])
        
        if (i + 1) % batch_size == 0:
            # extract features
            test_gold_dev.append(gold_deviation(buff))
            test_targets_value.append(get_targets_value(buff))
            test_items_value.append(get_items_value(buff))
            test_trends.append(timeseries_trend(buff))
            # free buff
            buff = []
    
    # process last batch
    test_gold_dev.append(gold_deviation(buff))
    test_targets_value.append(get_targets_value(buff))
    test_items_value.append(get_items_value(buff))
    test_trends.append(timeseries_trend(buff))

    # free buff
    buff = []

A Jupyter Widget

In [30]:
#columns_deviation
#columns_self_damage
#columns_friend_damage
#columns_tar_value
#columns_items_value
#trend_columns

In [41]:
def add_extra_feature(feature,columns_list):
    concated_feature = np.concatenate(feature)
    feature_df = pd.DataFrame(data=concated_feature, columns=columns_list)
    
    return feature_df

In [45]:
deviation_df = add_extra_feature(gold_dev,columns_deviation)

In [46]:
targets_df = add_extra_feature(targets_value,['targets_value'])

In [47]:
item_val_df = add_extra_feature(items_value,['item_val_df'])

In [48]:
trends_df = add_extra_feature(trends, columns_trend)

In [91]:
test_deviation_df = add_extra_feature(test_gold_dev,columns_deviation)

In [92]:
test_targets_df = add_extra_feature(test_targets_value,['targets_value'])

In [93]:
test_item_val_df = add_extra_feature(test_items_value,['item_val_df'])

## Apply Changes 

In [49]:
train = pd.concat([train, deviation_df, targets_df, item_val_df, trends_df],axis =1)

In [95]:
test = pd.concat([full_test, test_deviation_df,test_targets_df, test_item_val_df],axis=1)

## add hero data 

In [59]:
heroes = pd.read_csv('./sberbank_data/data_2019/dota2_heroes.csv', index_col='hero_id')

#### roles ====> one_hot

In [60]:
heroes['roles'] = heroes['roles'].apply(lambda x: x.replace(" ", '').strip('[]').replace("'", '').split(','))

In [61]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
heroes = heroes.join(pd.DataFrame(mlb.fit_transform(heroes.pop('roles')),
                          columns=mlb.classes_,
                          index=heroes.index))

#### attack_type, cm_enabled ====> one_hot

In [62]:
# get dummies
attack_type_dummies = pd.get_dummies(heroes['attack_type'], prefix='att_type_')
prim_attr_dummies = pd.get_dummies(heroes['primary_attr'], prefix='pr_attr_')

# drop colums
heroes.drop(['legs'],inplace=True, axis=1)
heroes.drop(['attack_type'], inplace=True, axis=1)
heroes.drop(['cm_enabled'], inplace=True, axis=1)
heroes.drop(['primary_attr'], inplace=True, axis=1)

# concatenate them to dataframe
heroes = pd.concat([heroes, 
                    attack_type_dummies, 
                    prim_attr_dummies], axis=1)

## Join heroes to data_train

In [63]:
heroes.columns

Index(['name', 'localized_name', 'agi_gain', 'attack_range', 'attack_rate',
       'base_agi', 'base_armor', 'base_attack_max', 'base_attack_min',
       'base_health', 'base_health_regen', 'base_int', 'base_mana',
       'base_mana_regen', 'base_mr', 'base_str', 'int_gain', 'move_speed',
       'projectile_speed', 'str_gain', 'turn_rate', 'Carry', 'Disabler',
       'Durable', 'Escape', 'Initiator', 'Jungler', 'Nuker', 'Pusher',
       'Support', 'att_type__Melee', 'att_type__Ranged', 'pr_attr__agi',
       'pr_attr__int', 'pr_attr__str'],
      dtype='object')

In [64]:
heroes.drop(['name', 'localized_name',], inplace=True, axis=1)

In [1]:
train = train.join(heroes, on='hero_id')

#apply one hot for 'hero_id'
#hero_dummies = pd.get_dummies(train['hero_id'], prefix='hero_id')
#train.drop(['hero_id'], inplace=True, axis=1)
#train = pd.concat([train, hero_dummies], axis=1)

In [103]:
test = full_test.join(heroes, on='hero_id')

#apply one hot for 'hero_id'
#hero_dummies = pd.get_dummies(test['hero_id'], prefix='hero_id')
#test.drop(['hero_id'], inplace=True, axis=1)
#test = pd.concat([test, hero_dummies], axis=1)

## Cross-val 

In [104]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [107]:
cat_feat = get_cat_index(train)

In [69]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
catb = CatBoostClassifier(learning_rate=0.1, l2_leaf_reg=15, max_depth=13,
                          cat_features=cat_feat, random_seed=1337, task_type='GPU',verbose=0,
                         bagging_temperature=1, random_strength=1,
                          custom_loss=['AUC', 'Accuracy'])

In [75]:
lgbm_clf = LGBMClassifier(learning_rate=0.025, 
                          n_estimators=2500, 
                          reg_lambda=11, num_leaves=69)

In [70]:
# learning_rate=0.05,l2_leaf_reg=15,max_depth=13,
#                           cat_features=cat_feat, random_seed=1421

In [144]:
cb_test = CatBoostClassifier(task_type='GPU',max_depth=12,reg_lambda=29)

In [78]:
from sklearn.model_selection import cross_validate

In [79]:
%time
scores = cross_validate(lgbm_clf,cv=StratifiedKFold(n_splits=3,random_state=1337),
                          X=train, y=y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


## Prediction 

In [60]:
def valid_index(train, n_folds = 3, r_state=228):
    all_index = list(train.index)
    shuffled_index = shuffle(all_index, random_state=r_state)
    samples = np.array_split(shuffled_index, n_folds)
    
    return samples

In [61]:
index = valid_index(train)

In [63]:
train_sample_one = train.iloc[index[0]]
train_sample_two = train.iloc[index[1]]
train_sample_three = train.iloc[index[2]]

y_train_sample_one = y_train.iloc[index[0]]
y_train_sample_two = y_train.iloc[index[1]]
y_train_sample_three = y_train.iloc[index[2]]

In [67]:
cb_best.fit(train_sample_one,y_train_sample_one,verbose=0,cat_features=cat_feat)

In [ ]:
xgb_best.fit(train,y_train,verbose=0)

In [235]:
lgbm_clf.fit(train, y_train)
#os.system('telegram-send "%s"' % msg);

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.28, max_depth=17,
        min_child_samples=40, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=3000, n_jobs=-1, num_leaves=31, objective=None,
        random_state=228, reg_alpha=0.0, reg_lambda=22, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [253]:
cb_pred = cb_best.predict_proba(test)
xgb_pred = xgb_best.predict_proba(test)
lgb_pred = lgbm_clf.predict_proba(test)

In [280]:
def weighted_pred(pred1,pred2,pred3,threshold):
    func = lambda x: x[1]
    
    pred1 = np.array(pred1)
    pred2 = np.array(pred2)
    pred3 = np.array(pred3)

    merge = (pred1+pred2+pred3) / 3
    merge = list(map(func, merge))
    predict = np.asarray(merge) > threshold
    
    return predict

In [138]:
pred = weighted_pred(cb_pred,xgb_pred,lgb_pred,threshold=0.40)

In [127]:
train.shape

(149202, 629)

In [126]:
test.shape

(43265, 774)

In [143]:
pred = catb.predict(test.drop(to_drop, axis=1))

## Save submit 

In [254]:
func = lambda x: x[1]

In [255]:
merge = list(map(func, lgb_pred))

In [258]:
pred = np.array(merge) > 0.55

In [144]:
predict = pd.DataFrame({'skilled': pred}, index=test_index)
predict.to_csv('cat_boost.csv')